In [27]:
import pandas as pd
from typing import List, Union, Dict
from transformers import pipeline
from tqdm import tqdm

books = pd.read_csv('books_with_categories.csv')

In [3]:
pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)


Device set to use cpu


In [4]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [18]:
# Analyze sentiment by whole sentences
sentences = books['description'][0].split('.')
# List of list of scores
predictions = pipe(sentences)
predictions




[[{'label': 'surprise', 'score': 0.7296021580696106},
  {'label': 'neutral', 'score': 0.14038598537445068},
  {'label': 'fear', 'score': 0.06816227734088898},
  {'label': 'joy', 'score': 0.04794260486960411},
  {'label': 'anger', 'score': 0.009156367741525173},
  {'label': 'disgust', 'score': 0.0026284768246114254},
  {'label': 'sadness', 'score': 0.002122163772583008}],
 [{'label': 'neutral', 'score': 0.4493706524372101},
  {'label': 'disgust', 'score': 0.27359113097190857},
  {'label': 'joy', 'score': 0.1090831384062767},
  {'label': 'sadness', 'score': 0.09362747520208359},
  {'label': 'anger', 'score': 0.04047833010554314},
  {'label': 'surprise', 'score': 0.02697019837796688},
  {'label': 'fear', 'score': 0.006879049818962812}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.242733433842659},
  {'label': 'disgust', 'score': 0.043422672897577286},
  {'label': 'surprise', 'score': 0.02830057218670845},
  {'label': 'joy', 'score': 0.01421149075

In [11]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [16]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296021580696106},
 {'label': 'neutral', 'score': 0.14038598537445068},
 {'label': 'fear', 'score': 0.06816227734088898},
 {'label': 'joy', 'score': 0.04794260486960411},
 {'label': 'anger', 'score': 0.009156367741525173},
 {'label': 'disgust', 'score': 0.0026284768246114254},
 {'label': 'sadness', 'score': 0.002122163772583008}]

In [15]:
# Add column for each emotion and assign the highest score as value
sorted(predictions[0], key=lambda x: x['label'])

[{'label': 'anger', 'score': 0.009156367741525173},
 {'label': 'disgust', 'score': 0.0026284768246114254},
 {'label': 'fear', 'score': 0.06816227734088898},
 {'label': 'joy', 'score': 0.04794260486960411},
 {'label': 'neutral', 'score': 0.14038598537445068},
 {'label': 'sadness', 'score': 0.002122163772583008},
 {'label': 'surprise', 'score': 0.7296021580696106}]

In [21]:
isbn = []

def calculate_max_emotion_scores(all_predictions: List[List[Dict[str, Union[str, float]]]]):
    max_scores = {}

    for prediction_list in all_predictions:
        for emotion in prediction_list:
            label = emotion['label']
            score = emotion['score']

            # Update max score if current score is higher
            if label not in max_scores or score > max_scores[label]:
                max_scores[label] = score

    return max_scores

max_emotion_scores = calculate_max_emotion_scores(predictions)
print(max_emotion_scores)

{'surprise': 0.7296021580696106, 'neutral': 0.6462159156799316, 'fear': 0.9281682968139648, 'joy': 0.9327981472015381, 'anger': 0.06413363665342331, 'disgust': 0.27359113097190857, 'sadness': 0.9671575427055359}


In [28]:
# Add emotional labels as col for first 10 books
for i in tqdm(range(len(books))):
    sentences = books['description'][i].split('.')
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)

    for label, score in max_scores.items():
        if label not in books.columns:
            books[label] = 0.0
        books.at[i, label] = score

100%|██████████| 5197/5197 [14:38<00:00,  5.92it/s]


In [29]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,surprise,neutral,fear,joy,anger,disgust,sadness
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.729602,0.646216,0.928168,0.932798,0.064134,0.273591,0.967158
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.252546,0.887940,0.942528,0.704422,0.612619,0.348284,0.111690
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.078765,0.549477,0.972321,0.767239,0.064134,0.104007,0.111690
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.078765,0.732685,0.360705,0.251882,0.351485,0.150722,0.111690
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.078765,0.884390,0.095043,0.040564,0.081412,0.184496,0.475880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.030656,0.853721,0.919165,0.255171,0.148209,0.030643,0.980877
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.227765,0.883198,0.051363,0.400263,0.064134,0.114383,0.111690
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.057625,0.375754,0.339218,0.947779,0.009997,0.009929,0.066685
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.078765,0.951104,0.459270,0.759457,0.064134,0.104007,0.368111


In [30]:
books.to_csv('books_with_emotions.csv', index=False)